# 10 Matching

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import style
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf

import graphviz as gr

%matplotlib inline

## Matching Estimator

- matching each treated unit with a similar untreated unit
- 그래서 treat group과 untreated group이 comparable하게 만드는 것

- 데이터를 이용하여 matching을 해보자.
- 아래의 데이터에서 `sex`, `age`, `severity`가 confounder이다.

In [5]:
med = pd.read_csv("./data/medicine_impact_recovery.csv")
med.head()

,sex,age,severity,medication,recovery
0,0,35.049134,0.887658,1,31
1,1,41.580323,0.899784,1,49
2,1,28.127491,0.486349,0,38
3,1,36.375033,0.323091,0,35
4,0,25.091717,0.209006,0,15


- 먼저 confounder를 고려하지 않으면 `medication=1`인 경우 16.9일 더 revovery 해야한다.

$$E[Y|T=1]-E[Y|T=0]$$

In [6]:
med.query("medication==1")["recovery"].mean() - med.query("medication==0")["recovery"].mean()

16.895799546498726

- mathcing을 하기 위해서 euclidean norm을 고려하자.
- 즉, 특정 instance의 반대 group에서 가장 가까운 하나의 instance를 골라서 causal effect를 구하는 것이다.
- 아래식은 Matching estimator이고
- $Y_{jm}$은 $Y_i$와 반대 group에 속하는 가장 가까운 instance이다.

$$\hat{ATE} = \frac{1}{N} \sum^N_{i=0} (2T_i - 1)\big(Y_i - Y_{jm}(i)\big)$$

In [8]:
from sklearn.preprocessing import StandardScaler

X = ["severity", "age", "sex"]
y = "recovery"

scaler = StandardScaler()
med[X] = scaler.fit_transform(med[X])

In [9]:
med.head()

,sex,age,severity,medication,recovery
0,-0.997004,0.280794,1.459836,1,31
1,1.003005,0.865397,1.502202,1,49
2,1.003005,-0.338758,0.057798,0,38
3,1.003005,0.399475,-0.512570,0,35
4,-0.997004,-0.610488,-0.911148,0,15


In [10]:
from sklearn.neighbors import KNeighborsRegressor

treated = med.query("medication==1")
untreated = med.query("medication==0")

mt0 = KNeighborsRegressor(n_neighbors=1).fit(untreated[X], untreated[y])
mt1 = KNeighborsRegressor(n_neighbors=1).fit(treated[X], treated[y])

predicted = pd.concat([
    # find matches for the treated looking at the untreated knn model
    treated.assign(match=mt0.predict(treated[X])),
    
    # find matches for the untreated looking at the treated knn model
    untreated.assign(match=mt1.predict(untreated[X]))
])

predicted.head()

,sex,age,severity,medication,recovery,match
0,-0.997004,0.280794,1.459836,1,31,39.0
1,1.003005,0.865397,1.502202,1,49,52.0
7,-0.997004,1.495171,1.268572,1,38,46.0
10,1.003005,-0.106537,0.545925,1,34,45.0
16,-0.997004,0.043035,1.428768,1,30,39.0


In [11]:
np.mean((2*predicted["medication"] - 1)*(predicted["recovery"] - predicted["match"]))

-0.9954

## Matching Bias

- Matching estimator를 biased되어 있다.
- 이에 대해 증명은 생략하고 알아보자.
- 먼저 $\hat{ATET}$를 아래와 같이 정의하자. (전개 과정을 편하게 하기 위해서 사용할 뿐, 아래 설명의 반대의 경우도 똑같이 사용해서 ATE 구할 수 있음)
  - $N_1$은 treated unit의 수
  - $Y_j(i)$는 the untreated match of treated unit $i$

$$\hat{ATET} = \frac{1}{N_1}\sum (Y_i - Y_j(i))$$

- $\mu_0(X_i)$: treated unit $i$가 treat되지 않았다면 outcome $Y$
- $\mu_0(X_j(i))$: $i$의 match인 untreated unit $j$의 outcome

$$E[\sqrt{N_1}(\hat{ATET} - ATET)] = E[\sqrt{N_1}(\mu_0(X_i) - \mu_0(X_j(i)))]$$

- CLT에 의해 위 값이 0으로 가야하는데 항상 그렇지는 않다고 한다.
- 위에서 비슷한 unit을 맞추더라도 완벽할 수 없고 그로 인해 차이가 생긴다고 이해할 수 있다.
- 전체 sample size가 많아질수록 match할 수 있는 unit들이 많아져서 그 간극이 줄어들지만 $\sqrt{N_1}$ 값이 커지는 속도가 더 빠르기 때문에 0으로 converge 못 할 수도 있다.

- 그렇다면 이런 bias를 보완할 수 있지 않을까?
- 아래 식처럼 예측모델을 이용하여 bias를 추정하여 그만큼 뻬주는 것이다.
- $\hat{\mu_0}(X_i)$을 예측모델을 이용하여 추정한다.

$$\hat{ATET} = \frac{1}{N_1}\sum \big((Y_i - Y_{j(i)}) - (\hat{\mu_0}(X_i) - \hat{\mu_0}(X_{j(i)}))\big)$$

In [12]:
from sklearn.linear_model import LinearRegression

# fit the linear regression model to estimate mu_0(x)
ols0 = LinearRegression().fit(untreated[X], untreated[y])
ols1 = LinearRegression().fit(treated[X], treated[y])

# find the units that match to the treated
treated_match_index = mt0.kneighbors(treated[X], n_neighbors=1)[1].ravel()

# find the units that match to the untreatd
untreated_match_index = mt1.kneighbors(untreated[X], n_neighbors=1)[1].ravel()

predicted = pd.concat([
    (treated
     .assign(match=mt0.predict(treated[X])) 
     .assign(bias_correct=ols0.predict(treated[X]) - ols0.predict(untreated.iloc[treated_match_index][X]))),

    (untreated
     .assign(match=mt1.predict(untreated[X]))
     .assign(bias_correct=ols1.predict(untreated[X]) - ols1.predict(treated.iloc[untreated_match_index][X])))
])

predicted.head()

,sex,age,severity,medication,recovery,match,bias_correct
0,-0.997004,0.280794,1.459836,1,31,39.0,4.404034
1,1.003005,0.865397,1.502202,1,49,52.0,12.915348
7,-0.997004,1.495171,1.268572,1,38,46.0,1.871428
10,1.003005,-0.106537,0.545925,1,34,45.0,-0.496970
16,-0.997004,0.043035,1.428768,1,30,39.0,2.610159


In [13]:
np.mean((2*predicted["medication"] - 1)*((predicted["recovery"] - predicted["match"])-predicted["bias_correct"]))

-7.362660906141412

## The Curse of Dimensionality

- means that the number of data points required to fill a feature space grows exponentially with the number of features, or dimensions
- 즉, feature가 많아질수록 가까이 있는 instance를 선택하더라도 실제로 꽤 먼 거리일 수도 있는 것이다.